In [1]:
import firebase_admin
from firebase_admin import credentials, db
import speech_recognition as sr
from googletrans import Translator

# ------------------------- Firebase Setup -------------------------
def initialize_firebase():
    # Replace with the path to your service account key JSON
    cred = credentials.Certificate("C:/Users/Sarthak/OneDrive/Desktop/BITS/service_account_key.json")

    if not firebase_admin._apps:  # Prevent re-initialization
        firebase_admin.initialize_app(cred, {
            'databaseURL': 'https://smart-box-4da35-default-rtdb.asia-southeast1.firebasedatabase.app'
        })

def get_patient_data():
    ref = db.reference('patient')
    data = ref.get()
    sos = data.get('SOS', 'No SOS Message')
    heart_rate = data.get('heartRate', 'No Heart Rate Data')
    temperature = data.get('temperature', 'No Temperature Data')
    return sos, heart_rate, temperature

# ------------------------- Speech Recognition -------------------------
def recognize_speech():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("🎤 बोलें... (Speak something)")
        recognizer.adjust_for_ambient_noise(source, duration=1)
        audio = recognizer.listen(source)

    try:
        detected_text = recognizer.recognize_google(audio, language="hi")
        print(f"🗣 पहचाना गया: {detected_text}")
        return detected_text
    except sr.UnknownValueError:
        print("❌ आवाज़ समझ नहीं आई (Could not understand audio)")
        return None
    except sr.RequestError:
        print("⚠ सर्वर से समस्या (Speech recognition service error)")
        return None

# ------------------------- Translation -------------------------
def translate_text(text):
    translator = Translator()
    translation = translator.translate(text, dest="en")
    print(f"✅ Translated: {translation.text}")
    return translation.text

# ------------------------- Prompt Generation -------------------------
def generate_prompt(sos, heart_rate, temperature, translated_speech):
    prompt = (
        f"The patient's vital signs are as follows:\n"
        f"- Heart Rate: {heart_rate} bpm\n"
        f"- Temperature: {temperature} °C\n"
        f"SOS Message: \"{sos}\"\n"
        f"Patient's spoken input (translated): \"{translated_speech}\"\n\n"
        "Based on these vitals and the patient's statement, what immediate medications should be taken?"
    )
    return prompt

# ------------------------- Main Function -------------------------
if __name__ == "__main__":
    initialize_firebase()

    # 1️⃣ Recognize speech and translate it
    speech_text = recognize_speech()
    translated_speech = translate_text(speech_text) if speech_text else "No speech input provided"

    # 2️⃣ Fetch patient data from Firebase
    sos, heart_rate, temperature = get_patient_data()

    # 3️⃣ Generate and display the final prompt
    final_prompt = generate_prompt(sos, heart_rate, temperature, translated_speech)
    print("\n📝 Generated Chatbot Prompt:\n")
    print(final_prompt)

AttributeError: module 'httpcore' has no attribute 'SyncHTTPTransport'